In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [18]:
doc_path = "docs\Final_HR Dept Process Document 2025.pdf"

In [19]:
loader = PyPDFLoader(file_path=doc_path)

In [20]:
hr_doc = loader.load()

In [17]:
hr_doc[7].page_content

"7 \n \n1.6  Institute Cadre as per RPR 2024 \nA brief overview of the Institute's cadres and levels is mentioned below. The comprehensive details are in the \nRecruitment and Promotion Rules for staff-2024  (Annexure I) \n  \nS. \nNo. \nName of \nCadres \nIncluded Departments \n1. General \nAdmin Cadre \nHR, Store & Purchase, Academics, Student Affairs, Department Admins, \nIRD Admin, Corporate Communication, Placement Office, Well -Being \nCell, Library, Resource Generation/Outreach, and Alumni Relations, etc. \n2. Accounts \nCadre \nInstitute and IRD Accounts \n3. Estate Cadre Estate, Infrastructure, Maintenance of Facilities, Housekeeping, Security, \nTransportation, Guest House, and Horticulture \n4. IT Cadre IT Support Services and website management \n5. Research \nCadre \nResearch support in laboratories \n \nCadre Distribution Pay Level Wise \n  \nInstitute Levels Pay Level \nSenior Levels Level-14 \nLevel-12 \nMiddle Levels Level-10 \nLevel-8 \nJunior Levels Level-6 \nLevel-4

In [21]:
hr_doc

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-04-21T10:57:10+05:30', 'author': 'aditi', 'moddate': '2025-04-21T10:57:10+05:30', 'source': 'docs\\Final_HR Dept Process Document 2025.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='INDRAPRASTHA INSTITUTE OF INFORMATION TECHNOLOGY-DELHI  \nHUMAN RESOURCE DEPARTMENT PROCESSES DOCUMENT (VERSION 1/2025)  \n(For the Use of the Non-Teaching Staff) \nHR DIVISION WILL ATTEMPT TO GIVE THEIR FIRST RESPONSE TO EMAILS WITHIN 2 WORKING DAYS. \n \nName of the Process Periodicity  How to \nrequest \n \nDocuments \nRequired \nTurnaround \ntime (Receipt in \nthe HR Dept \nWorking Days) \nWhom to Contact \n \nEffective 15 April \n2025 \nEscalation Matrix \nPoints of Escalation (PoE) \nPDA Processing Ongoing Filled claim \nform along with \ninvoice \nThe related form \nat the link : \nhttps://iiitd.ac.in/fo\nrm_docs \n4-5 working days Extn.-480 \nvinod@iiitd.ac.in \nSh. 

In [10]:
import pymupdf

In [1]:
## load manually, and extract all the text of the document 
from typing import List, Dict
import os
from pypdf import PdfReader
import re



def clean_text(text:str) -> str:
    return re.sub(r'[.…]{2,}|\s+', " ", text)

def load_pdf(file_path:str) -> List[Dict]:
    if not os.path.exists(file_path):
        return "Path Does not exist."
    else:
        pages = []
        reader = PdfReader(file_path)
        for i, page in enumerate(reader.pages):
            page_text = page.extract_text()
            if page_text:
                pages.append({
                    "page_no" : i,
                    "text" : clean_text(page_text).strip()
                })
        return pages







In [2]:
doc = load_pdf("docs\HR Handbook 2025 for website.pdf")

In [3]:
#now i have got the docs of all pages in the format [{'page_no' : page_no, 'text' : page_text}]
## I have to create chunking for this, which would take, the chunk size and chunk overlap


def chunking(doc:List[Dict], chunking_size:int = 500, overlap: int = 60)-> List[Dict]:
    full_page:str = ""
    char_to_page: List = []
    for page in doc:
        if full_page:
            full_page += " "
            char_to_page.append(page['page_no'])
        full_page += page['text']
        char_to_page.extend([page['page_no']]*len(page['text']))

    chunks: List[Dict] = []

    start = 0

    while start < len(full_page):
        end = min(start+500, len(full_page))
        chunk = full_page[start:end].strip()

        if chunk:
            page_no = sorted(set(char_to_page[start:end]))
        chunks.append({
            "chunk_text" : chunk,
            "page_no" : page_no
        })
        start += chunking_size - overlap

    return chunks





    




In [4]:
chunking(doc)

[{'chunk_text': 'HR HANDBOOK FOR STAFF Indraprastha Institute of Information Technology Delhi Okhla Industrial Estate Phase-III New Delhi – 110020 September 2025 2 TABLE OF CONTENTS Sr. No. Particulars Page No. 1) INTRODUCTION 1.1 Purpose of the Handbook 4 1.2 Scope and Applicability 4 1.3 Guiding Principle and Values 4 1.4 Overview of the Institute 4-5 1.5 Institutional & Reporting Structure 6 1.6 Institute Cadre as per RPR 2024 7 1.7 Institute Conduct Rules 7-9 2) JOINING 2.1 General Terms of Offer of Letter 1',
  'page_no': [1, 2]},
 {'chunk_text': 'Rules 7-9 2) JOINING 2.1 General Terms of Offer of Letter 10 2.2 Joining Forms 10 2.3 Joining Formalities and subsequent follow-up by HR 11 2.4 Fixation of Salary 11 2.5 Terms mentioned in the Service Contract Agreement 11-12 3) WORK CYCLE 3.1 Personal File & Service Book 13 3.2 Time & Attendance 13 3.3 Leave Entitlement 14-15 3.4 Promotion, Upgradation and Career Progression Process 15-16 3.5 Salary and Increments 16 3.6 National Pensio